In [ ]:
!git clone https://github.com/adarsh1001/Hand_Segmentation_RefineNet

Cloning into 'Hand_Segmentation_RefineNet'...
remote: Enumerating objects: 45, done.
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45
Unpacking objects: 100% (45/45), done.


In [ ]:
!cp /content/Hand_Segmentation_RefineNet/RefineNet/refinenet.py -r /content
!cp /content/Hand_Segmentation_RefineNet/RefineNet/weights -r /content
!cp /content/drive/MyDrive/handmask/hands.pth.tar -r /content/weights

In [ ]:
!pip install py7zr
import os
import glob
from pathlib import Path
from csv import writer
import cv2
from cv2 import VideoCapture, waitKey, imshow, cvtColor, flip, COLOR_BGR2RGB, COLOR_RGB2BGR, putText, FONT_HERSHEY_SIMPLEX, resize, rectangle, line, VideoWriter_fourcc, VideoWriter
from numpy import asscalar, ndarray, array, zeros, concatenate, load, uint8, set_printoptions, inf
from pathlib import Path
from shutil import rmtree
import py7zr
import random
from os.path import split
import matplotlib.pyplot as plt
import re
import torch
from refinenet import refinenet
from tqdm import tqdm

In [ ]:
with py7zr.SevenZipFile('/content/drive/MyDrive/Age,Bird,Favourite.7z', mode='r') as z:
    z.extractall('/content/ASLdataset')

In [ ]:
videos_per_class = 100
all_source_ds = []      #source video location list
source_directory = Path('/content/ASLdataset/Age,Bird,Favourite')
classes = ['Bird']
for category in classes:
    print(str(source_directory)+'/'+category)
    tempo_dir = random.sample(os.listdir(str(source_directory)+'/'+category), videos_per_class)
    cat_dir=[]
    for directory in tempo_dir:
        cat_dir.append(Path((source_directory/category/directory)))
    all_source_ds.append(cat_dir)
print(len(all_source_ds[len(classes)-1]))  #Remove len and GET READY FOR FLOOOD
counter_class = len(classes)
print(counter_class)
make_dir = Path('/content/Output')
if not make_dir.exists():make_dir.mkdir()
make_dir = Path('/content/temp')
if not make_dir.exists():make_dir.mkdir()

/content/ASLdataset/Age,Bird,Favourite/Bird
100
1


In [ ]:
def atoi(strr):
    return int(strr) if strr.isdigit() else strr

def natural_keys(strr):
    return [ atoi(c) for c in re.split('(\d+)', strr) ]
class Writer:
    temp_path = ''
    def write_to_csv(self, result: ndarray, out_csv: str):
        out_csv = Path(out_csv)
        with out_csv.open('a+') as csv:
            csv_writer = writer(csv)
            csv_writer.writerow(result)
    def write_as_video(self, path, image):
        if self.temp_path != path: 
            self.temp_path = path
            self.writer = VideoWriter(path, VideoWriter_fourcc('M','J','P','G'), 10, (640, 480))
        self.writer.write(image)  
w = Writer()
img_scale = 1./255
img_mean = array([0.485, 0.456, 0.406]).reshape((1, 1, 3))
img_std = array([0.229, 0.224, 0.225]).reshape((1, 1, 3))
numC = 7
cmap = load('weights/cmap.npy')
net = refinenet(numC, pretrained=True).eval()
net = net.cuda() #Run only with Cuda support!
pbar = tqdm(total=300, bar_format ='{l_bar}{bar}|  {n_fmt}/{total_fmt}  [ Elapsed Time : {elapsed} | ETA : {remaining} ]', leave=True, position=0)

  0%|          |  0/300  [ Elapsed Time : 00:00 | ETA : ? ]

In [ ]:
for i in range(0,1):
    current_ds = all_source_ds[i]
    label = split(split(current_ds[1])[0])[1]
    # make_dir = Path('/content/Output/'+label)
    # if not make_dir.exists():make_dir.mkdir()
    # print(label)
    for j in range(len(current_ds)):
        video = current_ds[j]
        video_cap = VideoCapture(str(video))
        flag, frame = video_cap.read()
        cnt =1
        frame_length = int(video_cap.get(cv2.CAP_PROP_FRAME_COUNT))  
        rmtree('/content/temp')
        Path('/content/temp').mkdir()
        while flag:
            cv2.imwrite('/content/temp/frame%d.jpg' %cnt, frame)
            flag, frame = video_cap.read()
            cnt+=1
        imgs = os.listdir('/content/temp')
        imgs.sort(key=natural_keys)
        for imgname in imgs:
            img = array(plt.imread('/content/temp/'+str(imgname)))
            orig_size = img.shape[:2][::-1]
            inter_img = (img*img_scale - img_mean)/img_std
            img_inp = torch.tensor(inter_img.transpose(2, 0, 1)[None]).float()
            img_inp = img_inp.cuda()
            seg = net(img_inp)[0].data.cpu().numpy().transpose(1, 2, 0)
            seg = cv2.resize(seg, orig_size, interpolation=cv2.INTER_CUBIC)
            seg = cmap[seg.argmax(axis=2).astype(uint8)]
            ma = (seg[:,:,0]>=0) & (seg[:,:,0]<=20) & (seg[:,:,2]>=110) & (seg[:,:,2]<=150) & (seg[:,:,1]>=0) & (seg[:,:,1]<=20)
            x = zeros(ma.shape)
            x[ma==True] = 1
            x[ma==False] = 0
            set_printoptions(threshold=inf)
            w.write_to_csv(x, '/content/Output/'+label+'_'+str(j)+'.csv')
        pbar.update(1)

Bird


 33%|███▎      |  100/300  [ Elapsed Time : 2:22:40 | ETA : 4:45:27 ]

In [ ]:
!zip -r /content/drive/MyDrive/handmask/OutputBird.zip /content/Output

  adding: content/Output/ (stored 0%)
  adding: content/Output/Bird_87.csv (deflated 99%)
  adding: content/Output/Bird_21.csv (deflated 99%)
  adding: content/Output/Bird_46.csv (deflated 99%)
  adding: content/Output/Bird_63.csv (deflated 99%)
  adding: content/Output/Bird_6.csv (deflated 99%)
  adding: content/Output/Bird_19.csv (deflated 99%)
  adding: content/Output/Bird_48.csv (deflated 99%)
  adding: content/Output/Bird_76.csv (deflated 99%)
  adding: content/Output/Bird_22.csv (deflated 99%)
  adding: content/Output/Bird_55.csv (deflated 99%)
  adding: content/Output/Bird_82.csv (deflated 99%)
  adding: content/Output/Bird_52.csv (deflated 99%)
  adding: content/Output/Bird_18.csv (deflated 99%)
  adding: content/Output/Bird_47.csv (deflated 99%)
  adding: content/Output/Bird_31.csv (deflated 99%)
  adding: content/Output/Bird_2.csv (deflated 99%)
  adding: content/Output/Bird_84.csv (deflated 99%)
  adding: content/Output/Bird_12.csv (deflated 99%)
  adding: content/Output/Bir

In [ ]:
!cp /content/3class.zip -r /content/drive/MyDrive/handmask

In [ ]:
from google.colab import files
files.download('/content/3class.zip') 

In [ ]:
from pandas import DataFrame, read_csv, get_dummies, Series, concat
csv_list = []
source_directory = '/content/Output'
videos_per_class = 200
classes = [ 'Bird' ]
for category in classes:
    print(str(source_directory)+'/'+category)
    tempo_dir = random.sample(os.listdir(str(source_directory)), videos_per_class)
    cat_dir=[]
    for directory in tempo_dir:
        if 'Favourite' in directory:
            cat_dir.append(Path((source_directory+'/'+directory)))
    csv_list.append(cat_dir)
print(len(csv_list[0]))
print(csv_list)

/content/Output/Bird
100
[[PosixPath('/content/Output/Favourite_8.csv'), PosixPath('/content/Output/Favourite_16.csv'), PosixPath('/content/Output/Favourite_28.csv'), PosixPath('/content/Output/Favourite_96.csv'), PosixPath('/content/Output/Favourite_5.csv'), PosixPath('/content/Output/Favourite_61.csv'), PosixPath('/content/Output/Favourite_69.csv'), PosixPath('/content/Output/Favourite_39.csv'), PosixPath('/content/Output/Favourite_47.csv'), PosixPath('/content/Output/Favourite_23.csv'), PosixPath('/content/Output/Favourite_70.csv'), PosixPath('/content/Output/Favourite_71.csv'), PosixPath('/content/Output/Favourite_86.csv'), PosixPath('/content/Output/Favourite_49.csv'), PosixPath('/content/Output/Favourite_31.csv'), PosixPath('/content/Output/Favourite_38.csv'), PosixPath('/content/Output/Favourite_18.csv'), PosixPath('/content/Output/Favourite_50.csv'), PosixPath('/content/Output/Favourite_62.csv'), PosixPath('/content/Output/Favourite_44.csv'), PosixPath('/content/Output/Favourit

In [ ]:
csv_list = csv_list[0]

In [ ]:
import shutil
destination = Path('/content/Outputfav')
count1 = 0
for item in csv_list:
    if destination.exists():
        count1+=1
        shutil.move(str(item), str(destination))

print(count1)

100


In [ ]:
!zip -r /content/drive/MyDrive/handmask/Outputfav.zip /content/Outputfav

In [ ]:
from refinenet import refinenet
import cv2

import matplotlib
matplotlib.use('agg') #Tk not working on Ada
import matplotlib.pyplot as plt

import numpy as np
import torch
import os
import re

cmap = np.load('weights/cmap.npy')
img_dir = './'
numC = 7

img_scale = 1./255
img_mean = np.array([0.485, 0.456, 0.406]).reshape((1, 1, 3))
img_std = np.array([0.229, 0.224, 0.225]).reshape((1, 1, 3))

net = refinenet(numC, pretrained=True).eval()
net = net.cuda() #Run only with Cuda support!
idx = 1

def atoi(strr):
    return int(strr) if strr.isdigit() else strr

def natural_keys(strr):
    return [ atoi(c) for c in re.split('(\d+)', strr) ]

with torch.no_grad():
    for i in range(len(all_source_ds)):
        video = all_source_ds[i]
        i+=1
        video_cap = cv2.VideoCapture(video)
        flag, image = video_cap.read()
        cnt = 1

        
        os.chdir('frames/')

        while flag:
            cv2.imwrite('frame%d.jpg' %cnt, image)
            flag, image = video_cap.read()
            cnt += 1

        imgs = os.listdir('.')
        imgs.sort(key=natural_keys)

        for imgname in imgs:
            img = np.array(plt.imread(imgname))
            orig_size = img.shape[:2][::-1]

            inter_img = (img*img_scale - img_mean)/img_std

            img_inp = torch.tensor(inter_img.transpose(2, 0, 1)[None]).float()
            img_inp = img_inp.cuda()

            seg = net(img_inp)[0].data.cpu().numpy().transpose(1, 2, 0)
            seg = cv2.resize(seg, orig_size, interpolation=cv2.INTER_CUBIC)
            seg = cmap[seg.argmax(axis=2).astype(np.uint8)]

            ma = (seg[:,:,0]>=0) & (seg[:,:,0]<=20) & (seg[:,:,2]>=110) & (seg[:,:,2]<=150) & (seg[:,:,1]>=0) & (seg[:,:,1]<=20)
            x = np.zeros(ma.shape)
            x[ma==True] = 255
            x[ma==False] = 0
            cv2.imwrite('../segmented_frames/'+imgname,x)
            idx += 1
        os.chdir('../segmented_frames')
        frames = os.listdir('.')
        frames.sort(key=natural_keys)
        img = cv2.imread(frames[0])
        h,w,l = img.shape
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter('../segmented_video/segmented_video.mp4', fourcc, 24.0, (w, h))
        for frame in frames:
            frame = cv2.imread(frame)
            out.write(frame)
        out.release()


